In [3]:
import pandas as pd

# Parquet 파일 경로
parquet_file_path = r'C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq.parquet'

# Parquet 파일의 첫 몇 개의 행을 불러오기
df_head = pd.read_parquet(parquet_file_path, engine='pyarrow')

print(df_head.head())   # 데이터의 첫 몇 개의 행을 출력하여 구조 확인
print(df_head.info())   # 데이터의 기본 정보(열 이름, 데이터 타입 등)를 확인



                                      context_before           equation  \
0  The horizontal axis represents the exponent range  $ \in [ 3 , 7 ] $   
1                                               Here          $ d\eta $   
2                               The agent 's actions        $ a ( t ) $   
3  are modeled by a stochastic control process wi...     $ \sigma^2_t $   
4                                               Here        $ \deltat $   

                                       context_after  eq_type paper_number  \
0  . We selected those categorical colors from Co...        0   2301.00002   
1  denotes the system noise , modeled as a Wiener...        0   2301.00005   
2  are modeled by a stochastic control process wi...        0   2301.00005   
3  controlled by the agent and with a mean of zer...        0   2301.00005   
4  denotes the physical time step , and we adopte...        0   2301.00005   

  paper_type  equation_len  
0  arxiv2023            17  
1  arxiv2023          

In [6]:
import pandas as pd
import re
import json
import time
from tqdm import tqdm

def load_patterns(filename):
    """JSON 파일에서 패턴 로드"""
    with open(filename, 'r') as file:
        return json.load(file)

def apply_patterns(latex_str, patterns):
    """패턴을 적용하여 LaTeX 수식을 자연어로 변환"""
    for pattern in patterns:
        regex_pattern = pattern['pattern']
        replacement = pattern['replacement']
        latex_str = re.sub(regex_pattern, lambda m: eval(f'f"{replacement}"'), latex_str)
    return latex_str

def latex_to_english(latex_str, patterns):
    """LaTeX 수식을 자연어로 변환"""
    latex_str = latex_str.replace(r'\frac', 'frac')
    return apply_patterns(latex_str, patterns)

def is_fully_converted(latex_str):
    """수식이 온전히 변환되었는지 여부를 확인"""
    # 변환되지 않은 LaTeX 패턴이 남아 있는지 확인
    return 0 if re.search(r'\\[a-zA-Z]+', latex_str) else 1

# JSON 파일에서 패턴 로드
patterns = load_patterns('patterns.json')

# Parquet 파일 경로
parquet_file_path = r'C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq.parquet'

# Parquet 파일의 데이터를 한 번에 불러오기
df = pd.read_parquet(parquet_file_path, engine='pyarrow')

# 변환 작업 전 시간 기록
start_time = time.time()

# tqdm을 사용하여 진행률 표시 및 로그 추가
equation_en = []
complete = []
for equation in tqdm(df['equation'], desc="Processing equations", unit="equations"):
    english_equation = latex_to_english(equation, patterns)
    equation_en.append(english_equation)
    complete.append(is_fully_converted(english_equation))

# 새로운 열에 변환 결과 및 변환 여부 추가
df['equation_en'] = equation_en
df['complete'] = complete

# 변환 작업 후 시간 기록 및 처리 속도 계산
end_time = time.time()
elapsed_time = end_time - start_time
speed = len(df) / elapsed_time

# complete 열의 0과 1의 비율 계산 및 출력
complete_counts = df['complete'].value_counts(normalize=True) * 100
print(f"Percentage of fully converted equations (1): {complete_counts.get(1, 0):.2f}%")
print(f"Percentage of partially converted equations (0): {complete_counts.get(0, 0):.2f}%")

# 완전히 변환된 데이터와 부분적으로 변환된 데이터를 분리하여 저장
df_complete_1 = df[df['complete'] == 1]
df_complete_0 = df[df['complete'] == 0]

output_parquet_path_1 = r'C:\github\MathBridge\spoken_LaTex\Data\df_complete_1.parquet'
output_parquet_path_0 = r'C:\github\MathBridge\spoken_LaTex\Data\df_complete_0.parquet'

df_complete_1.to_parquet(output_parquet_path_1, engine='pyarrow', index=False)
df_complete_0.to_parquet(output_parquet_path_0, engine='pyarrow', index=False)

# 로그 출력
print(f"Transformation complete and saved to {output_parquet_path_1} and {output_parquet_path_0}")
print(f"Total time taken: {elapsed_time:.2f} seconds")
print(f"Processing speed: {speed:.2f} equations per second")


Processing equations: 100%|██████████| 12052508/12052508 [07:03<00:00, 28436.20equations/s]


Transformation complete and saved to C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq_en.parquet
Total time taken: 424.82 seconds
Processing speed: 28370.55 equations per second


In [7]:
import pandas as pd

# Parquet 파일 경로
parquet_file_path = r'C:\github\MathBridge\spoken_LaTex\Data\df_not_len_5_cleaned_unique_eq_en.parquet'

# Parquet 파일의 첫 몇 개의 행을 불러오기
df_head = pd.read_parquet(parquet_file_path, engine='pyarrow')

print(df_head.head())   # 데이터의 첫 몇 개의 행을 출력하여 구조 확인
print(df_head.info())   # 데이터의 기본 정보(열 이름, 데이터 타입 등)를 확인

                                      context_before           equation  \
0  The horizontal axis represents the exponent range  $ \in [ 3 , 7 ] $   
1                                               Here          $ d\eta $   
2                               The agent 's actions        $ a ( t ) $   
3  are modeled by a stochastic control process wi...     $ \sigma^2_t $   
4                                               Here        $ \deltat $   

                                       context_after  eq_type paper_number  \
0  . We selected those categorical colors from Co...        0   2301.00002   
1  denotes the system noise , modeled as a Wiener...        0   2301.00005   
2  are modeled by a stochastic control process wi...        0   2301.00005   
3  controlled by the agent and with a mean of zer...        0   2301.00005   
4  denotes the physical time step , and we adopte...        0   2301.00005   

  paper_type  equation_len                               equation_en  
0  arxiv2